In [1]:
from google.colab import drive
drive.mount('/content/driver')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/driver


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from tensorflow import keras
import warnings
from glob import glob
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

warnings.filterwarnings("ignore")

In [3]:

# data = pd.read_csv('/content/driver/My Drive/Deep-Learning/stock3/data_stocks.csv')
path_train = '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/*.csv'
files = glob(path_train)
x_input =[]
y_output=[]
print(files)
for file in files:
  data = pd.read_csv(file)
  data_at = data['AT'].values
  data_xt = data['XT'].values
  data_ac = data['AC'].values/100
  data_total_ac = data['TOTAL_AC'].values/100
  data_total_at = data['TOTAL_AT'].values

#dung de lay du lieu theo cap
  data_input3D = []
  data_output2D = []
  steps = 3
  
  for i in range(0,len(data_xt)-steps+1):
    temp_3D = []
    temp_output_2D = []
    for k in range(i, i+steps):
      temp_2D = []
      temp_2D.append(data_at[k])
      temp_2D.append(data_xt[k])
      temp_2D.append(data_ac[k])
      
      temp_3D.append(temp_2D)

    temp_output_2D.append(data_total_at[i])
    temp_output_2D.append(data_total_ac[i])

    data_input3D.append(temp_3D)
    data_output2D.append(temp_output_2D)

  x_input.append(data_input3D)
  y_output.append(data_output2D)

x_input = np.concatenate( x_input, axis=0 )
y_output = np.concatenate( y_output, axis=0 )
x_train = np.asarray(x_input)
y_train = np.asarray(y_output)

x_train = x_train.reshape(-1,3,3)
y_train = y_train.reshape(-1,2)


['/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataA.csv', '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataC.csv', '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataE.csv', '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataF.csv', '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataD.csv', '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataG.csv', '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/train_k59/dataB.csv']


In [0]:
n_input = 3
n_features = 3

model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_input, n_features)))
model.add(RepeatVector(3))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(2)))
adam = Adam(lr=0.0005) 
model.compile(loss='mse', optimizer='adam')
 

In [11]:
model.fit(x_train, y_train, epochs=1200, validation_split=0.2, verbose=1, batch_size=3)
model.save('/content/driver/My Drive/Deep-Learning/LSTM_Software/Code/Use Data_K59/ATXTAC_DOAN_TOTALAT_TOTALAC/3_1_ac.h5')

ValueError: ignored

In [0]:
model = keras.models.load_model('/content/driver/My Drive/Deep-Learning/LSTM_Software/Code/Use Data_K59/ATXTAC_DOAN_TOTALAT_TOTALAC/3_1_ac.h5')
path_test = '/content/driver/My Drive/Deep-Learning/LSTM_Software/data/k59/datacsv/test_k59/dataH.csv'

test = pd.read_csv(path_test)
ip_test_at = test['AT'].values.astype('float64')
ip_test_xt = test['XT'].values.astype('float64') 
ip_test_ac = test['AC'].values.astype('float64')/100
test_output_total_ac = test['TOTAL_AC'].values.astype('float64')
test_output_total_at = test['TOTAL_AT'].values.astype('float64')


x_input=[]
y_output =[]

data_input3D = []
data_output2D = []
steps = 3
for i in range(0,len(ip_test_at)-steps+1):
  temp_3D = []
  temp_output_2D = []
  for k in range(i, i+steps):
    temp_2D = []
    temp_2D.append(ip_test_at[k])
    temp_2D.append(ip_test_xt[k])
    temp_2D.append(ip_test_ac[k])
    
    temp_3D.append(temp_2D)

  temp_output_2D.append(test_output_total_at[i])
  temp_output_2D.append(test_output_total_ac[i])

  data_input3D.append(temp_3D)
  data_output2D.append(temp_output_2D)

x_input.append(data_input3D)
y_output.append(data_output2D)

x_test = np.asarray(x_input)
y_test = np.asarray(y_output)



print(x_test.shape)
x_test = x_test.reshape(-1,3,3)
y_test = y_test.reshape(-1,2)

print(x_test.shape)

result = model.predict(x_test)


print(result)
print(y_test)
# result = result*100
# # result = scaler.inverse_transform(result)

print(result[:,1]*100)
plt.plot(result[:,1]*100,'x', color='r')
plt.plot(y_test[:,1], 'o' ,color='b')
plt.title("Mô hình K60")
plt.xlabel("STT")
plt.ylabel("TOTAL_AC")
# mplcursors.cursor(hover=True)
print("actually: ",y_test)
print("model: ", result)
plt.legend(('prediction', 'reality'),loc='upper right')
plt.show()